In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install d2l==0.17.6

In [ ]:
import os
path = '/content/drive/MyDrive'
os.chdir(path)

!source venv_d2l/bin/activate

path = '/content/drive/MyDrive/d2l-zh'
os.chdir(path)

数值稳定性是机器学习里特别重要的一点。特别是神经网络层数很深的时候，数值特别容易不稳定。

因为我们做了太多的矩阵乘法，所以有问题，可能会导致梯度爆炸或梯度消失。

梯度爆炸的问题：例如使用ReLU做激活函数。值超过浮点运算变成infinity，对于16位浮点数尤为严重（使用GPU的时候会使用16位浮点数，6e-5 - 6e4，因为nVidia的GPU16位浮点数会比32位浮点数快2倍，16位浮点数最大的问题是数值区间很低）。

假设没有到无穷大，还是会有很多问题。最大的问题是他对学习率eta非常敏感。如果学习率稍微大一点点，就会带来比较大的参数的值，因为我每一步走得比较远，对权重的更新就比较大。权重一大，梯度的乘法就大，就会带来更大的梯度。更大的梯度会导致更大的参数值，迭代个几回，整个梯度就炸掉了。

假设学习率太小，每一次对w的增加比较小，训练没有进展。

很有可能我们要在训练中不断调整学习率，因为大一点就炸掉了，小一点不动，就只有很小的一个范围可以调学习率，就比较难调。

梯度消失的问题：例如使用sigmoid做激活函数。因为输出的值在0到1之间，看梯度话，如果大于6，梯度就基本上到0了。

梯度消失最严重的问题是梯度值变成0，对16位浮点数尤为严重，小于5e-4基本上就可以当0看了。当梯度值变成0，那么不管怎么学你的学习率都不会有进展。因为你的权重是学习率乘以你的梯度，不管学习率取多大，因为梯度已经是0了，训练就不会有进展。

对于比较深的网络的时候，对于底部层尤为严重。梯度反传的时候是从顶开始的，顶部第一层就一次矩阵乘法，那么梯度可能是正常的，越到下面梯度可能很小了，底部那些层，就是靠数据近的那些层可能就是0了。

底部层跑不动，只把顶部层训练好，这就意味和很浅的神经网络没有区别。

总结：当数值过大或者过小时会导致数值问题。常发生在深度模型中，因为其会对n个数累乘。

让训练更加稳定

目标：让梯度值在合理的范围内，例如[1e-6, 1e3].

将乘法变加法：ResNet, LSTM

归一化：梯度归一化，梯度裁剪

这一章要讲的是如何合理的权重初始化和激活函数。


让每层的方差是一个常数。

将每层的输出和梯度都看做随机变量。
让他们的均值和方差都保持一致。

一般是均值为0，方差保持为一个常数。
这是一个假设，我们希望设计神经网络，以达到这样的性质。

权重初始化

在合理值区间里面随机初始参数。这是因为训练开始的时候更容易有数值不稳定。远离最优解的地方损失函数表面可能很复杂。最优解附近表面会比较平。

使用N(0, 0.01)来初始可能对小网络没问题，但不能保证深度神经网络。

Xavier初始

难以需要满足 n_(t-1) * gamma_t = 1 and n_t * gamma_t = 1.
对于t层，n_(t-1)是输入个数，n_t是输出个数，除非输入对于输出个数。

那么怎么做呢？做一点权衡，不能满足同时，那么就做一个折中。Xaiver使得 gamma_t * (n_(t-1) + n_t) / 2 = 1, gamma_t = 2 / (n_(t-1) + n_t).
正态分布 N(0, sqrt(2/(n_(t-1) + n_t)))
均匀分布 U(-sqrt(6/(n_(t-1) + n_t)), sqrt(6/(n_(t-1) + n_t)))，分布U[-a,a]和方差是a^2/3. 适配权重形状变换，特别是n_t.

检查常用激活函数

使用泰勒展开，tanh和relu在零点附近是近似到f(x)=x, identity函数。权重通常是在零点附近比较小的数。sigmoid不行，需要调整为4*sigmoid(x)-2. tanh和relu效果不错也是有数值稳定性的原因。scaled sigmoid也不错。

合理的权重初始值和激活函数的选取可以提升数值稳定性。


inf通常是初始值太大或者learning rate太大导致的。nan通常是除以0造成的。合理初始化权重，激活函数不要选错，学习率不要太大，可以解决。

梯度太小的话就是平的，train不动，没什么进展。nan就是梯度爆炸导致的。

bfloat16和32位很接近。

整个深度学习的进展都是让数值更加稳定。


In [ ]:
# 梯度消失
%matplotib inline
import torch
from d2l import torch as d2l

x = torch.arange(-8.0, 8.0, 0.1, requires_grad=True)
y = torch.sigmod(x)
y.backward(torch.ones_like(x))

d2l.plot(x.detach().numpy(), [y.detach().numpy(), x.grad.numpy()],
         legend=['sigmoid', 'gradient'], figsize=(4.5, 2.5))

In [ ]:
M = torch.normal(0, 1, size=(4,4))
print('一个矩阵 \n', M)
for i in range(100):
  M = torch.mm(M, torch.normal(0, 1, size=(4, 4)))

print('乘以100个矩阵后\n', M)